<a href="https://colab.research.google.com/github/iGhostlp/Albus/blob/severus/Proyecto_BBVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Armado Entorno

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [ ]:
ls

sample_data/  spark-3.3.2-bin-hadoop3.tgz


In [ ]:
!tar xf spark-3.3.2-bin-hadoop3.tgz


In [ ]:
!java -version

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [ ]:
!readlink -f $(which java) | sed "s:bin/java::"


/usr/lib/jvm/java-11-openjdk-amd64/


In [ ]:
!echo $JAVA_HOME

In [ ]:
!pip install -q findspark


In [ ]:
!pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=1245a01828d775a544e7bbe0d97006c9d0e32e461a1ca9becd604012afe3d630
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'

In [ ]:
import findspark


In [ ]:
import pandas as pd


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
findspark.init()


In [ ]:
findspark.find()


'/content/spark-3.3.2-bin-hadoop3'

In [ ]:
from pyspark.sql import SparkSession


In [ ]:
from pyspark.sql.functions import col


In [ ]:
spark = SparkSession.builder.master("local[*]").config('spark.sql.parquet.datetimeRebaseModeInRead','CORRECTED').getOrCreate()


In [ ]:
spark

## Extraccion de datos desde .parquet, clientes y direcciones.

In [ ]:
df_address = spark.read.parquet('/content/Datasets/address.snappy.parquet')

AnalysisException: ignored

In [ ]:
df_customer_address = df_address.join(df_customer, 'customer_id')

### Filtrado Address

In [ ]:
df_address_cut = df_address.drop('address_priority_number','address_sequence_id','residence_type','address_without_number_type','province_id','address_country_id','other_information_desc','address_relationship_type','address_start_date','address_verified_date','customer_locator_verified_type','address_status_mod_date','contact_channel_type','sender_application_id','returned_mail_type','normalization_status_type','normalization_reason_name','normalization_date','normalized_level_match_number','dlvy_day_monday_type','dlvy_day_tuesday_type','dlvy_day_wednesday_type','dlvy_day_thursday_type','dlvy_day_friday_type','dlvy_day_saturday_type','delivery_contact_start_hm_date','delivery_contact_end_hm_date','prev_address_sequence_id','registry_entry_date','register_user_id','last_change_user_id','last_change_hms_date','last_change_terminal_id','registration_type','operational_load_date')

In [ ]:
df_address_cut.toPandas()

In [ ]:
df_address_sorted = df_address_cut.orderBy([df_address_cut.customer_id, desc('last_change_date')])
df_address_sorted.toPandas()

In [ ]:
df_address_sorted = df_address_sorted.select('customer_id', concat(df_address_sorted.street_name,df_address_sorted.address_outdoor_id,df_address_sorted.address_indoor_id,df_address_sorted.indoor_number).alias('Full_Address'),'address_district_name','address_town_name','address_department_name','zipcode_id','long_zipcode_id','last_change_date')

In [ ]:
df_address_sorted.toPandas()

In [ ]:
window = Window.partitionBy(df_address_sorted.customer_id).orderBy(desc(df_address_sorted.last_change_date))

In [ ]:
df_address = df_address_sorted.withColumn('row_num', row_number().over(window))

In [ ]:
df_address = df_address.filter(df_address.row_num <= 3)
df_address.toPandas()

In [ ]:
df_pivot_address = df_address.groupBy('customer_id').agg(collect_list('Full_Address').alias('last_3_changes_list'))

In [ ]:
df_pivot_address = df_pivot_address.selectExpr('customer_id', 'last_3_changes_list[0] as Address_1', 'last_3_changes_list[1] as Address_2', 'last_3_changes_list[2] as Address_3')

In [ ]:
df_pivot_address = df_pivot_address.na.fill('---')
df_pivot_address.toPandas()